In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
  temperature=0.1,
)

memory = ConversationBufferMemory(return_messages=True)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"
    ),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
  ]
)

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = (
  {
    "context": retriver,
    "question": RunnablePassthrough(),
    "history": RunnableLambda(load_memory),
  }
  | prompt
  | llm
)

def invoke_chain(question):
  result = chain.invoke(question)
  memory.save_context(
    {"input": question},
    {"output": result.content},
  )
  print(result)

In [2]:
invoke_chain("Aaronson은 유죄인가요?")

content='제가 알기로는 Aaronson이 유죄임을 확실히 알 수 없습니다.'


In [3]:
invoke_chain("그가 테이블에 어떤 메세지를 썼나요?")

content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'


In [4]:
invoke_chain("Julia는 누구인가요?")

content='Julia는 주인공과 함께 이야기하는 여성 캐릭터입니다.'
